# Assignment 3 - Quantum Communication 
### <span style="color:#61c2e8">Task 1</span>: BB84 Key Distribution

In class you have learned about the protocols for exchanging classical and quantum units of information. BB84 is one such protocol. BB84 is a protocol proposed by Charles Bennet and Gilles Brassard in 1984. It was introduced for securely generating random bitstrings, which can then be used to encrypt information to securely share it between parties. In this task we are going to look at this protocol, and how its behavior changes as we introduce noise to its operation.

#### Run these cells before beginning to import necessary packages.
This cell may take a minute or more to complete. Importing qsharp triggers a sequence of actions which allow us to compile and run Q# code in this notebook.

In [1]:
import qsharp
import qsharp.azure
import matplotlib.pyplot as plt

Preparing Q# environment...
.

In [2]:
%%qsharp 
open Microsoft.Quantum.Diagnostics; 
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Random;

The first step of BB84 is for the sender to prepare the qubits by initializing them randomly to $|0\rangle, |1\rangle, |+\rangle \text{ or } |-\rangle$. These basis states represent two choices done by the sender: the bit they want to encode (0 or 1) and the basis they encode this bit in. 
Remember that $|0\rangle$ and $|1\rangle$ are the basis vectors in the Z basis and $|+\rangle$ and $|-\rangle$ are the basis vectors in the X basis.

The operation `Sender` represents the first part of the protocol, the actions taken by the sender.

In [3]:
%%qsharp
/// # Summary
/// Returns a random bit (0 or 1) with equal probability.
///
/// # Output
/// A random integer in {0, 1}

operation RandomBit() : Int {
    return DrawRandomInt(0, 1);
}

/// # Summary
/// This operation prepares each of the qubits in the input array in one of the |0⟩, |1⟩, |+⟩, or |-⟩ states randomly.
/// Additionally, it returns a tuple of two arrays, an array of the chosen bases (X or Z) and an array of the encoded bits (0 or 1).
/// 
/// # Input
/// ## qs
/// A qubit array that the sender prepares randomly in X and Z basis as either 0 or 1
///
/// # Output
/// A tuple of two integer arrays. Each element of the first array indicates the basis chosen, 0 for Z, 1 for X.
/// The corresponding element of the second array indicates whether the qubit was prepared as 0 or 1 in that basis.

operation Sender(qs : Qubit[]) : (Int[], Int[]) {
    // Create the array where we track the chosen basis
    mutable chosenBases = [];
    mutable encodedBits = [];
    for i in 0 .. Length(qs) - 1 {
        // 0 - 0 in chosen basis, 1 - 1 in chosen basis
        set encodedBits = encodedBits + [RandomBit()];
        if encodedBits[i] == 1 {
            X(qs[i]); // 0 to 1
        }
        // 0 - Z basis, 1 - X basis
        set chosenBases = chosenBases + [RandomBit()]; 
        if chosenBases[i] == 1 {
            H(qs[i]); //Swap our basis
        }
    }
    // In the BB84 protocol, the sender will only communicate the bases to the receiver. 
    // In our implementation we're also returning the bits encoded to allow for protocol visualization.
    return (chosenBases, encodedBits);
}

After this, we implement the second part of the protocol, the actions done by the receiver after receiving the qubits prepared by the sender. 


In [4]:
%%qsharp
/// # Summary
/// Performs the receiver side of key distribution. 
/// Measures each qubit in a randomly chosen basis, X or Z, and returns the chosen bases and the measurement results.
/// 
/// # Input
/// ## qs
/// Array of qubits which were prepared by the sender
///
/// # Output
/// A tuple of two integer arrays. Each element of the first array indicates the basis chosen for measurement, 0 for Z, 1 for X.
/// The corresponding element of the second array indicates whether the qubit was measured as 0 or 1 in that basis.

operation Receiver(qs : Qubit[]) : (Int[], Int[]) {
    mutable measuredBases = [];
    for i in 0 .. Length(qs) - 1 {
        // We need to randomly select a base to measure in
        set measuredBases = measuredBases + [RandomBit()];
        if measuredBases[i] == 1 {
            H(qs[i]);
        }
    }
    let decodedBits = ForEach((x) => M(x) == One ? 1 | 0, qs);
    return (measuredBases, decodedBits);
}

So now we just need to call our operations in the right order to perform key distribution. The length of generated keys is probabilistic, since we do not know how many times the sender and the receiver will choose the same basis. Our BB84 operation takes an argument $N$ which specifies the number of protocol iterations we run. The length of our key will be between 0 and N, and on average it will be $\frac{1}{2}N$.

In [5]:
%%qsharp 

/// # Summary
/// This operation acts as the intermediary exchanging
/// both classical and quantum information between parties.
/// N controls the number of qubits transmitted, so our final
/// key will be of length less than or equal to N.
///
/// # Input
/// ## N
/// The number of qubits to attempt key distribution with.
/// 
/// # Output
/// A tuple of 5 integer arrays. 
/// 1 - The preparation bases the sender used.
/// 2 - The measurement bases the receiver used.
/// 3 - The bits that the sender prepared in either basis.
/// 4 - The bits that were measured by the receiver.
/// 5 - The key that the receiver made after comparing bases.

operation BB84(N : Int) : (Int[], Int[], Int[], Int[], Int[]) {
    use qs = Qubit[N];
    let (basesS, bitS) = Sender(qs);
    let (basesR, bitR) = Receiver(qs);
    // Now we can reconstruct the key
    mutable key = [];
    for i in 0 .. N - 1 {
        if basesR[i] == basesS[i] {
            // Append the measurement result in integer form to key
            set key += [bitR[i]];
        } 
    }

    // Return some other things for analysis
    return (basesS, basesR, bitS, bitR, key);
}

Let's analyze things in Python a bit now.

In [6]:
def formatOutput(basesS, basesR, bitS, bitR, key):
    keyCopy = key.copy()
    same = "|" 
    basisSentChar = "|"
    basisRecChar = "|"
    bitSent = "|"
    bitRec = "|"
    keyS = "|"
    keyR = "|"
    stateSent = "|"
    for i in range(len(basesR)):
        bitSent += ' 1 |' if bitS[i] == 1 else ' 0 |'
        bitRec += ' 1 |' if bitR[i] == 1 else ' 0 |'
        same += ' y |' if basesR[i] == basesS[i] else ' n |'
        keyS += ' {} |'.format(bitS[i]) if basesR[i] == basesS[i] else "   |"
        keyR += ' {} |'.format(key.pop(0)) if basesR[i] == basesS[i] else "   |"
        basisRecChar += ' Z |' if basesR[i] == 0 else ' X |'
        if basesS[i] == 0:
            stateSent += "|0>|" if bitS[i] == 0 else "|1>|"
            basisSentChar += " Z |"
        else:
            stateSent += "|+>|" if bitS[i] == 0 else "|->|"
            basisSentChar += " X |"

    print("Let's compare this to the selected bases, and the transmitted qubit states")
    print("Our sender bases were:      {}".format(basisSentChar))
    print("The receiver bases were:    {}".format(basisRecChar))
    print("Both bases match (yes/no):  {}".format(same))
    print("The sender encoded bit:     {}".format(bitSent))
    print("The states sent were:       {}".format(stateSent))
    print("The receiver measured:      {}".format(bitRec))
    print("Notice how the key is formed by the bits where bases are equal")
    print("Receiver key:               {}".format(keyR))
    print("Sender key:                 {}".format(keyS))
    print("The key that was generated was {}\n".format(keyCopy))
(basesS, basesR, bitS, bitR, key) = BB84.simulate(N=20)
formatOutput(basesS, basesR, bitS, bitR, key)

Let's compare this to the selected bases, and the transmitted qubit states
Our sender bases were:      | Z | Z | X | X | Z | Z | X | X | Z | Z | Z | X | X | X | Z | X | Z | X | X | X |
The receiver bases were:    | X | Z | X | Z | X | Z | X | X | X | X | X | X | X | X | X | Z | Z | Z | X | Z |
Both bases match (yes/no):  | n | y | y | n | n | y | y | y | n | n | n | y | y | y | n | n | y | n | y | n |
The sender encoded bit:     | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 0 | 1 | 1 | 1 | 0 |
The states sent were:       ||0>||0>||->||->||1>||0>||->||->||1>||0>||1>||+>||->||+>||1>||+>||1>||->||->||+>|
The receiver measured:      | 0 | 0 | 1 | 1 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 1 | 0 |
Notice how the key is formed by the bits where bases are equal
Receiver key:               |   | 0 | 1 |   |   | 0 | 1 | 1 |   |   |   | 0 | 1 | 0 |   |   | 1 |   | 1 |   |
Sender key:                 |   | 0 | 1 |   |   | 0 | 1 | 1 |   |   |   | 0 | 1 | 0 |   |   

In an ideal scenario we will transmit keys over a noiseless channel with no eavesdroppers. So it is not necessary to check whether the key that has been transmitted is correct. In the real world this may not be true. 

We can modify our transmission channel by adding an operation in BB84 between the sender and receiver which can add noise and act as an unwanted observer. Let's call this operation ObserverAndNoise(). It will apply an X operation to each qubit with probability, $p$. If we set the observer flag to true it will measure the qubit and apply some strategy for replicating the state.

In [7]:
%%qsharp

/// # Summary
/// This operation adds in a controllable noise source and an observer to the 
/// key distribution signal chain.
///
/// # Input
/// ## qs
/// The qubit array that is being transmitted.
/// ## p
/// The probability of applying our noise model, in [0, 1]
/// ## observer
/// A Boolean which indicates whether an observer is present.

operation ObserverAndNoise(qs : Qubit[], p : Double, observe : Bool) : Unit {
    // First, we will add a really simple noise model
    for q in qs { 
        // Flip qubit with probability p
        if DrawRandomBool(p) {
            X(q); 
        }
        if observe {
            // Measure the qubit
            let result = M(q);
            // Apply observer's strategy for replicating the state they've intercepted.
            // In this case, just send the state that matches the measurement result by 
            // doing nothing to it, since the measurement leaves it in the measured state. 
        }
    }
}

In [8]:
%%qsharp

/// # Summary
/// A top level operation for BB84 protocol which incorporates the observer and noise.
///
/// # Input
/// ## N
/// The number of qubits to attempt key distribution with.
/// ## p
/// The probability of an error occurring due to the channel noise.
/// ## observer
/// Indicates the presence of an observer that measures each qubit in the Z basis.
/// 
/// # Output
/// A tuple of 5 integer arrays. 
/// 1 - The preparation bases the sender used.
/// 2 - The measurement bases the receiver used.
/// 3 - The bits that the sender prepared in either basis.
/// 4 - The key that the receiver made after comparing bases.
/// 5 - The key that the receiver made after comparing bases.

operation BB84_Noisy(N : Int, p : Double, observe : Bool) : (Int[], Int[], Int[], Int[], Int[]) {
    use qs = Qubit[N];
    let (basesS, bitS) = Sender(qs);

    ObserverAndNoise(qs, p, observe);

    let (basesR, bitR) = Receiver(qs);
    mutable key = [];
    // Now we construct the key
    for i in 0 .. N - 1 {
        if basesR[i] == basesS[i] {
            // Append the measurement result in integer form to key
            set key += [bitR[i]];
        } 
    }
    // Now return some helpful data for analysis.
    return (basesS, basesR, bitS, bitR, key);
}

In [17]:
import numpy as np

obs_levels = [True, False]
p_levels = np.array(range(10, 80, 5)) * (1/100)
print(obs_levels)
print(p_levels)

[True, False]
[0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75]


In [34]:
# (basesS, basesR, bitS, bitR, key) = BB84_Noisy.simulate(N=20, p=0.3, observe=True)
# formatOutput(basesS, basesR, bitS, bitR, key)

# file_s = open("Task 1", "w+")
# output = ""

# for obs_level in obs_levels:
#     print("****** obs_level: " + str(obs_level) + " [START] ******")
#     for p_level in p_levels:
#         (basesS, basesR, bitS, bitR, key) = BB84_Noisy.simulate(N=20, p=p_level, observe=True)
#         print("****** (p_level, obs_leve): (" + str(p_level) + ", " + str(obs_level) + ") ******")
#         formatOutput(basesS, basesR, bitS, bitR, key)

p_level=0
for iter in range(0, 10):
    print("****** val of iter is: " + str(iter) + " ******")
    for obs_level in obs_levels:
        print("****** obs_level: " + str(obs_level) + " [START] ******")
        (basesS, basesR, bitS, bitR, key) = BB84_Noisy.simulate(N=20, p=p_level, observe=True)
        print("****** (p_level, obs_leve): (" + str(p_level) + ", " + str(obs_level) + ") ******")
        formatOutput(basesS, basesR, bitS, bitR, key)

# file_s.write(output)
# file_s.close()

****** val of iter is: 0 ******
****** obs_level: True [START] ******
****** (p_level, obs_leve): (0, True) ******
Let's compare this to the selected bases, and the transmitted qubit states
Our sender bases were:      | X | X | Z | X | X | X | X | X | X | Z | X | Z | Z | Z | X | X | Z | X | X | X |
The receiver bases were:    | Z | X | Z | X | X | X | Z | Z | X | Z | Z | X | Z | Z | X | Z | Z | Z | X | Z |
Both bases match (yes/no):  | n | y | y | y | y | y | n | n | y | y | n | n | y | y | y | n | y | n | y | n |
The sender encoded bit:     | 1 | 0 | 0 | 1 | 1 | 1 | 0 | 1 | 0 | 0 | 1 | 1 | 0 | 0 | 1 | 1 | 0 | 1 | 0 | 1 |
The states sent were:       ||->||+>||0>||->||->||->||+>||->||+>||0>||->||1>||0>||0>||->||->||0>||->||+>||->|
The receiver measured:      | 0 | 0 | 0 | 0 | 1 | 1 | 0 | 0 | 1 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 1 | 0 | 0 |
Notice how the key is formed by the bits where bases are equal
Receiver key:               |   | 0 | 0 | 0 | 1 | 1 |   |   | 1 | 0 |   |   | 0 | 0 | 0

### <span style="color:#61c2e8">Task 1 Questions</span>:
* T1.1. Experiment with noise levels and turning the observer on and off. In the results, can you distinguish the noise from the observer?

* T1.2. Assuming there is no noise, how can the sender and the receiver identify the presence of the observer?

##### T1.1

To experiment with noise levels and how the observer plays a role in BB84 communication methods, I have simulated for *p* values from *p*$=0.1$ to *p*$=0.75$ in steps of $0.05$ with the effects of the observer and without the effects of the observer. I ran this simulation over $10$ times. The results of $10$ of the simulations are shown graphically below.

![Simulation 1](https://raw.githubusercontent.com/aniruddha121500/ECE59500_proj/main/simulation_1.png)

![Simulation 2](https://raw.githubusercontent.com/aniruddha121500/ECE59500_proj/main/simulation_2.png)

![Simulation 3](https://raw.githubusercontent.com/aniruddha121500/ECE59500_proj/main/simulation_3.png)

![Simulation 4](https://raw.githubusercontent.com/aniruddha121500/ECE59500_proj/main/simulation_4.png)

![Simulation 5](https://raw.githubusercontent.com/aniruddha121500/ECE59500_proj/main/simulation_5.png)

![Simulation 6](https://raw.githubusercontent.com/aniruddha121500/ECE59500_proj/main/simulation_6.png)

![SImulation 7](https://raw.githubusercontent.com/aniruddha121500/ECE59500_proj/main/simulation_7.png)

![Simulation 8](https://raw.githubusercontent.com/aniruddha121500/ECE59500_proj/main/simulation_8.png)

![Simulation 9](https://raw.githubusercontent.com/aniruddha121500/ECE59500_proj/main/simulation_9.png)

![Simulation 10](https://raw.githubusercontent.com/aniruddha121500/ECE59500_proj/main/simulation_10.png)

As seen by the simulation outputs above, it is very hard to distinguish an observer from noise. WHen we are guranteed that the channel does not have any noise and we observe a disturbance in the communication channel then we have reason to believe that there is an unwanted observer in the communication channel. Nevertheless, this relationship is not very evident from the simulated runs shown above.

##### T1.2

As mentioned above, if we are guranteed that the communication channel does not have any noise then one way to know if there is an unwanted observer is to check for noise. In a noiseless communication channel, if there is noise between the sender and recieverm then the noise is generated by an unwanted observer. Although this relationship is not very evident from the simulated runs shown above, theoretically, the presence of an observer should induce more noise into the cimmunication channle than if the unwanted observer was absent.

For task 2 of this assignment we will be moving to a new notebook file. Please now open Assignment_3_Task2 to begin. 